# Загрузка модулей:

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

import sys
sys.path.append('./src')

In [ ]:
from ipy_utils import (IPYInteractiveSegmentation, show_video,
                       IPYRadioButtons, IPYButton)
from sam2al import SAM2
from cvat import CVATLabels
from cvat_srv import CVATSRV
from utils import TimeIt

# Открытие видеофайла и списка классов

In [ ]:
video_file = './project/videos/14_fragment_60.avi'

with TimeIt('загрузку модели') as t:
    sam2 = SAM2('~/models/sam2.1_hiera_large.pt')

with TimeIt('загрузку видео') as t:
    sam2.load_video(video_file,
                    prompts_file='./project/prompts',
                    offload_video_to_cpu=True,
                    offload_state_to_cpu=False,
                    async_loading_frames=False)

In [ ]:
labeles = CVATLabels('./project/labeles.txt')
IPYRadioButtons(**labeles.get_ipy_radio_buttons_kwargs('Класс объекта:'),
                setter=sam2.set_label).show();

# Интерактивная сегментация:

In [ ]:
sam2.go_to(frame=411, obj_id=9)
ipis = IPYInteractiveSegmentation(**sam2.init_ipis_kwargs())

# Группировка определённых классов:

In [ ]:
for obj_id in sam2.prompts.get_all_obj_ids():
    if sam2.prompts.get_label(obj_id) not in ['People', 'transport', 'field']:
        sam2.prompts.set_group(obj_id, 1)
sam2.prompts.make_auto_save()

# Таблица подсказок:

In [ ]:
df = sam2.prompts.df

# Комментируй ненужное:

# selected_df = df[df['track_id'] == 6]     # Отобразить все подсказки заданного трека
# selected_df = df[df['label'] == 'glade']  # Отобразить все подсказки заданного класса
# selected_df = df[df['frame'] == 10]       # Отобразить все подсказки заданного кадра
selected_df = df                          # Отобразить все подсказки

# selected_df = selected_df.head(50)  # Берём верхние 50 записей
selected_df = selected_df.tail(50)  # Берём нижние 50 записей

selected_df

# Сборка превью:

In [ ]:
with TimeIt('формирование превью с подгонкой контуров') as t:
    video = show_video(sam2.render_preview('DraftPreview.mp4',
                                           fit_segments=False), size=480)
video

In [ ]:
with TimeIt('формирование превью с подгонкой контуров') as t:
    video = show_video(sam2.render_preview('CleanPreview.mp4',
                                           fit_segments=True), size=480)
video

# Создание XML-файла:

In [ ]:
from IPython.display import display, HTML
import base64

In [ ]:
# Разметка:

xml_file = sam2.save2cvat_xml()
with open(xml_file, "rb") as file:
    xml_encoded = base64.b64encode(file.read()).decode('utf-8')
HTML(f'<a download="annotation.xml" href="data:text/xml;base64,{xml_encoded}" target="_blank">annotation.xml</a>')

In [ ]:
# Видео:

video_name = os.path.basename(sam2.video_file)
with open(sam2.video_file, "rb") as file:
    video_encoded = base64.b64encode(file.read()).decode('utf-8')
HTML(f'<a download="{video_name}" href="data:application/avi;base64,{video_encoded}" target="_blank">{video_name}</a>')

# Экспорт в CVAT:

In [ ]:
# Этую ячейку достаточно запустить один раз за сессию.

cvat_srv = CVATSRV('http://89.169.168.184:8080')
print('Соединение с', str(cvat_srv), 'установлено.\n')

proj_name = '2024_Krasnodarskiy_krai'
proj = cvat_srv[proj_name]

In [ ]:
with TimeIt('cоздание/обновление задачи в CVAT') as t:
    url = sam2.export2cvat(proj)
url